# Lab 1.4.1 Solution: NumPy Neural Network from Scratch

This notebook contains complete solutions to the exercises from Lab 1.4.1.

---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(42)
%matplotlib inline

## Exercise 1 Solution: Different Architectures

Let's compare different network architectures.

In [ ]:
# Complete MLP implementation
class MLP:
    def __init__(self, layer_sizes):
        self.layers = []
        for i in range(len(layer_sizes) - 1):
            W = np.random.randn(layer_sizes[i], layer_sizes[i + 1]) * np.sqrt(2.0 / layer_sizes[i])
            b = np.zeros(layer_sizes[i + 1])
            self.layers.append({'W': W, 'b': b, 'cache': {}, 'dW': None, 'db': None})
    
    def forward(self, X):
        out = X
        for i, layer in enumerate(self.layers[:-1]):
            layer['cache']['X'] = out
            out = out @ layer['W'] + layer['b']
            layer['cache']['Z'] = out
            out = np.maximum(0, out)
        
        self.layers[-1]['cache']['X'] = out
        out = out @ self.layers[-1]['W'] + self.layers[-1]['b']
        out_shifted = out - np.max(out, axis=1, keepdims=True)
        exp_out = np.exp(out_shifted)
        self.probs = exp_out / np.sum(exp_out, axis=1, keepdims=True)
        return self.probs
    
    def backward(self, targets, lr):
        batch_size = len(targets)
        grad = self.probs.copy()
        grad[np.arange(batch_size), targets] -= 1
        
        for i in range(len(self.layers) - 1, -1, -1):
            layer = self.layers[i]
            X = layer['cache']['X']
            layer['dW'] = X.T @ grad / batch_size
            layer['db'] = np.mean(grad, axis=0)
            grad = grad @ layer['W'].T
            if i > 0:
                Z = self.layers[i - 1]['cache']['Z']
                grad = grad * (Z > 0)
            layer['W'] -= lr * layer['dW']
            layer['b'] -= lr * layer['db']
    
    def predict(self, X):
        return np.argmax(self.forward(X), axis=1)

# Load MNIST (simplified)
import gzip
import urllib.request
import os

def load_mnist(path='../data'):
    os.makedirs(path, exist_ok=True)
    base_url = 'http://yann.lecun.com/exdb/mnist/'
    files = ['train-images-idx3-ubyte.gz', 'train-labels-idx1-ubyte.gz',
             't10k-images-idx3-ubyte.gz', 't10k-labels-idx1-ubyte.gz']
    
    for f in files:
        fp = os.path.join(path, f)
        if not os.path.exists(fp):
            urllib.request.urlretrieve(base_url + f, fp)
    
    def load_img(fp): 
        with gzip.open(fp) as f: f.read(16); return np.frombuffer(f.read(), np.uint8).reshape(-1,784).astype(np.float32)/255
    def load_lbl(fp): 
        with gzip.open(fp) as f: f.read(8); return np.frombuffer(f.read(), np.uint8)
    
    return (load_img(os.path.join(path, files[0])), load_lbl(os.path.join(path, files[1])),
            load_img(os.path.join(path, files[2])), load_lbl(os.path.join(path, files[3])))

X_train, y_train, X_test, y_test = load_mnist()

In [ ]:
# Compare architectures
architectures = {
    'Simple [784, 128, 10]': [784, 128, 10],
    'Standard [784, 256, 128, 10]': [784, 256, 128, 10],
    'Wide [784, 512, 512, 10]': [784, 512, 512, 10],
    'Deep [784, 256, 256, 128, 64, 10]': [784, 256, 256, 128, 64, 10]
}

def train_and_evaluate(arch, epochs=5, lr=0.1, batch_size=64):
    np.random.seed(42)
    model = MLP(arch)
    
    for epoch in range(epochs):
        indices = np.random.permutation(len(X_train))
        for start in range(0, len(X_train), batch_size):
            batch_idx = indices[start:start+batch_size]
            model.forward(X_train[batch_idx])
            model.backward(y_train[batch_idx], lr)
    
    return np.mean(model.predict(X_test) == y_test)

print("Architecture Comparison (5 epochs):")
print("=" * 60)
for name, arch in architectures.items():
    acc = train_and_evaluate(arch)
    params = sum((arch[i] + 1) * arch[i+1] for i in range(len(arch)-1))
    print(f"{name:40s} | Acc: {acc:.2%} | Params: {params:,}")

## Exercise 2 Solution: Learning Rate Exploration

In [ ]:
# Compare learning rates
learning_rates = [0.001, 0.01, 0.1, 1.0]
results = {}

for lr in learning_rates:
    np.random.seed(42)
    model = MLP([784, 256, 128, 10])
    history = []
    
    for epoch in range(10):
        indices = np.random.permutation(len(X_train))
        epoch_loss = 0
        
        for start in range(0, len(X_train), 64):
            batch_idx = indices[start:start+64]
            probs = model.forward(X_train[batch_idx])
            loss = -np.mean(np.log(probs[np.arange(len(batch_idx)), y_train[batch_idx]] + 1e-10))
            if not np.isnan(loss):
                epoch_loss += loss
            model.backward(y_train[batch_idx], lr)
        
        acc = np.mean(model.predict(X_test[:1000]) == y_test[:1000])
        history.append(acc)
    
    results[lr] = history

# Plot
plt.figure(figsize=(10, 5))
for lr, history in results.items():
    plt.plot(history, label=f'LR={lr}', linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Test Accuracy')
plt.title('Learning Rate Comparison')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("\nFinal Accuracies:")
for lr, history in results.items():
    status = "✅ Good" if history[-1] > 0.9 else "❌ Too high/low"
    print(f"  LR={lr}: {history[-1]:.2%} {status}")

## Challenge Solution: SGD with Momentum

In [ ]:
class MLPWithMomentum:
    """MLP with SGD Momentum optimizer."""
    
    def __init__(self, layer_sizes, momentum=0.9):
        self.layers = []
        self.momentum = momentum
        
        for i in range(len(layer_sizes) - 1):
            W = np.random.randn(layer_sizes[i], layer_sizes[i + 1]) * np.sqrt(2.0 / layer_sizes[i])
            b = np.zeros(layer_sizes[i + 1])
            self.layers.append({
                'W': W, 'b': b, 'cache': {},
                'vW': np.zeros_like(W),  # Velocity for weights
                'vb': np.zeros_like(b)   # Velocity for bias
            })
    
    def forward(self, X):
        out = X
        for i, layer in enumerate(self.layers[:-1]):
            layer['cache']['X'] = out
            out = out @ layer['W'] + layer['b']
            layer['cache']['Z'] = out
            out = np.maximum(0, out)
        
        self.layers[-1]['cache']['X'] = out
        out = out @ self.layers[-1]['W'] + self.layers[-1]['b']
        out_shifted = out - np.max(out, axis=1, keepdims=True)
        exp_out = np.exp(out_shifted)
        self.probs = exp_out / np.sum(exp_out, axis=1, keepdims=True)
        return self.probs
    
    def backward(self, targets, lr):
        batch_size = len(targets)
        grad = self.probs.copy()
        grad[np.arange(batch_size), targets] -= 1
        
        for i in range(len(self.layers) - 1, -1, -1):
            layer = self.layers[i]
            X = layer['cache']['X']
            
            dW = X.T @ grad / batch_size
            db = np.mean(grad, axis=0)
            
            # Update velocity with momentum
            layer['vW'] = self.momentum * layer['vW'] - lr * dW
            layer['vb'] = self.momentum * layer['vb'] - lr * db
            
            # Update weights using velocity
            layer['W'] += layer['vW']
            layer['b'] += layer['vb']
            
            grad = grad @ layer['W'].T
            if i > 0:
                Z = self.layers[i - 1]['cache']['Z']
                grad = grad * (Z > 0)
    
    def predict(self, X):
        return np.argmax(self.forward(X), axis=1)

# Compare with and without momentum
print("Comparing SGD vs SGD with Momentum:")
print("=" * 60)

# Without momentum
np.random.seed(42)
model_no_mom = MLP([784, 256, 128, 10])
for epoch in range(5):
    indices = np.random.permutation(len(X_train))
    for start in range(0, len(X_train), 64):
        batch_idx = indices[start:start+64]
        model_no_mom.forward(X_train[batch_idx])
        model_no_mom.backward(y_train[batch_idx], 0.1)
acc_no_mom = np.mean(model_no_mom.predict(X_test) == y_test)

# With momentum
np.random.seed(42)
model_mom = MLPWithMomentum([784, 256, 128, 10], momentum=0.9)
for epoch in range(5):
    indices = np.random.permutation(len(X_train))
    for start in range(0, len(X_train), 64):
        batch_idx = indices[start:start+64]
        model_mom.forward(X_train[batch_idx])
        model_mom.backward(y_train[batch_idx], 0.1)
acc_mom = np.mean(model_mom.predict(X_test) == y_test)

print(f"SGD (no momentum): {acc_no_mom:.2%}")
print(f"SGD (momentum=0.9): {acc_mom:.2%}")
print(f"\n💡 Momentum typically helps with faster convergence!")

---

## Key Takeaways

1. **Architecture matters**: Wider/deeper networks have more capacity but need more data
2. **Learning rate is critical**: Too high = unstable, too low = slow
3. **Momentum accelerates training**: Standard choice is 0.9
4. **He initialization** is important for ReLU networks